In [5]:
import logging 
#Importing libraries 
logging.basicConfig(filename='Log_Example.log',level=logging.DEBUG)
logging.info("Libraries importing started")
import numpy as np
import pandas as pd
import cx_Oracle
import dateutil
import datetime as dt 
import xgboost as xgb
import lightgbm as lgb
import seaborn as sns
color = sns.color_palette()
import matplotlib
import matplotlib.pyplot as plt 
from datetime import datetime as dt 
from sklearn.preprocessing import Imputer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier



logging.info("Libraries importing completed")


logger=logging.getLogger(__name__)
def dbConnection(connectString):
    logging.info("DB connection started %s",connectString)
    connection = cx_Oracle.connect(connectString)
    logger.info("DB connection successfull %s",connection)
    logging.info
    return connection 

def get_raw_data(connection,queryString,start,end):
    logging.info(start)
    df = pd.read_sql(queryString,con=connection)
    logging.info(end)
    return df 

def transform_df(df):
    logTransformation = ['V_OT_TXN_AMOUNT_LKR','V_RA_TXN_AMOUNT_LKR','V_MC_TXN_AMOUNT_LKR','V_TT_TXN_AMOUNT_LKR','V_SCH_PMNT_AMOUNT_LKR','PTVALUE',
                        'ATM_TRXN_BILL_AMT','POS_TRXN_BILL_AMT','FD_AMOUNT_SUM','INACTV_LOAN_LIM_TOT','MAX_LOAN_LIM',
                        'MIN_LOAN__LIM','C','D','vishwaTxAmt']
    df[logTransformation] = df[logTransformation].apply(lambda x : np.log(x+1),axis=1)
    
    #sqrtLogTransformation = []
    #df[logTransformation] = df[sqrtLogTransformation].apply(lambda x : np.sqrt(np.log(np.abs(x+1))),axis=1)
    
    sqrtTransformation = ['V_TXN_CNT','Dr2Cr']
    df[sqrtTransformation] = df[sqrtTransformation].apply(lambda x: np.sqrt(np.abs(x)),axis=1)

    return(df)

vishCustomer='select a.USER_ID,CUST_ACCOUNT,NIC_NUMBER,CUST_REF_NO, BANK_CUST_FLG,CUST_GENDER AS V_GENDER , \
CIVIL_STATUS AS V_CIVIL_STATUS, \
TO_CHAR(CUST_DOB) AS V_CUST_DOB, \
COMM_CITY AS V_CITY,CHARGE_PROFILE, \
case when CUST_CCARD is not null then \'Yes\' else \'No\' END as CREDIT_CARD_FLAG, \
case when lower(comm_country) like \'%sri%\' then \'Yes\' else \'No\' end as V_COUNTRY_SL, \
round((LAST_DAY(add_months(trunc(sysdate,\'mm\'),-1))-activate_date),0) as VISHWA_AGE, \
round((LAST_DAY(add_months(trunc(sysdate,\'mm\'),-1))-last_login_date),0) as LAST_LOGIN_AGE \
from TBAADM.svr_user_master_table a \
WHERE R_STATUS=104 '

vishTransfer='select USER_ID, \
SUM(CASE WHEN TRAN_MODE=\'OT\' then TXN_AMOUNT_LKR ELSE 0 END) AS V_OT_TXN_AMOUNT_LKR, \
SUM(CASE WHEN TRAN_MODE=\'RA\' then TXN_AMOUNT_LKR ELSE 0 END) AS V_RA_TXN_AMOUNT_LKR, \
SUM(CASE WHEN TRAN_MODE=\'RT\' then TXN_AMOUNT_LKR ELSE 0 END) AS V_RT_TXN_AMOUNT_LKR, \
SUM(CASE WHEN TRAN_MODE=\'MC\' then TXN_AMOUNT_LKR ELSE 0 END) AS V_MC_TXN_AMOUNT_LKR, \
SUM(CASE WHEN TRAN_MODE=\'TT\' then TXN_AMOUNT_LKR ELSE 0 END) AS V_TT_TXN_AMOUNT_LKR \
,count(*)AS V_TXN_CNT \
from tbaadm.svr_transfers_master_table \
where tran_date>= add_months(trunc(sysdate,\'mm\'),-6) AND tran_date<= LAST_DAY(add_months(trunc(sysdate,\'mm\'),-1)) \
group by USER_ID \
ORDER BY USER_ID'

vishPayment='SELECT CUST_ID as USER_ID,sum(TRANSACTION_AMT_LKR) AS V_PMNT_TRANSACTION_AMT_LKR,count(*) AS V_PMNT_COUNT \
FROM tbaadm.svr_payments_master_table \
where tran_date>= add_months(trunc(sysdate,\'mm\'),-6) AND tran_date<= LAST_DAY(add_months(trunc(sysdate,\'mm\'),-1)) \
group by CUST_ID \
ORDER BY USER_ID'

vishSchdPayment='SELECT USER_ID, sum(TXN_AMOUNT_LKR) AS V_SCH_PMNT_AMOUNT_LKR,count(*) V_SCH_PMNT_CNT \
FROM tbaadm.svr_schedule_pmt_tfr_table \
where SCH_DATE>= add_months(trunc(sysdate,\'mm\'),-6) AND SCH_DATE<= LAST_DAY(add_months(trunc(sysdate,\'mm\'),-1)) \
group by USER_ID \
ORDER BY USER_ID'

userCustMap='SELECT USER_ID, \
CUST_ID, \
COUNT(*) OVER (PARTITION BY USER_ID) U_Cnt, \
COUNT(*) OVER (PARTITION BY CUST_ID) C_Cnt \
FROM tbaadm.svr_cust_id_table'

custQuery='select crm.core_cust_id as CUST_ID,lower(crm.uniqueidnumber),crm.gender,crm.staffflag,crm.card_holder, \
(EXTRACT(YEAR FROM SYSDATE)-crm.birth_year) CUST_AGE \
,crm.cust_type,crm.occupation,crm.city,(LAST_DAY(add_months(trunc(sysdate,\'mm\'),-1))-crm.relationshipopeningdate) BANK_CUST_AGE_DAYS, \
crm.seniorcitizen,crm.blacklisted,crm.negated,crm.suspended \
from crmuser.accounts crm \
inner join (select distinct cust_id from tbaadm.svr_cust_id_table) svr on crm.core_cust_id=svr.cust_id'

pawnCustomer='select lower(a.PTCUSID) AS P_NIC_NUMBER,SUM(case when ptstatus=\'R\' then 1 else 0 end) PTSTATUS_R, \
SUM(case when ptstatus=\'A\' then 1 else 0 end) PTSTATUS_A,SUM(case when ptstatus=\'C\' then 1 else 0 end) PTSTATUS_C, \
sum(PTDECLAREDVALUE) as PTVALUE \
from tbaadm.pawnticket a \
inner join tbaadm.advance b on a.ptnum=b.advptnum \
GROUP BY lower(a.ptcusid)'

txnATMQuery='select b.cust_id AS CUST_ID,SUM(bill_amt) ATM_TRXN_BILL_AMT \
from stg.stg_swt_dx_trxn_all a \
inner join tbaadm.gam@stagelink b on a.orgn_acct_id=b.foracid \
where merc_cat_code in (\'6011\',\'6010\') and stat_code=1 and orgn_acct_id <> \'NA\' \
and added_date>= add_months(trunc(sysdate,\'mm\'),-6) AND added_date<= LAST_DAY(add_months(trunc(sysdate,\'mm\'),-1)) \
GROUP BY b.cust_id'

txnPOSQuery='select b.cust_id AS CUST_ID,SUM(bill_amt) POS_TRXN_BILL_AMT \
from stg.stg_swt_dx_trxn_all a \
inner join tbaadm.gam@stagelink b on a.orgn_acct_id=b.foracid \
where merc_cat_code not in (\'6011\',\'6010\')  and orgn_acct_id <> \'NA\' \
AND merc_cat_code <> \'NA\' and merc_cat_code is not null \
and added_date>= add_months(trunc(sysdate,\'mm\'),-6) AND added_date<= LAST_DAY(add_months(trunc(sysdate,\'mm\'),-1)) \
GROUP BY b.cust_id'

fdQuery='select a.cust_id, \
sum(case when acct_cls_flg=\'Y\' then 1 else 0 end) FD_CLS_CNT,sum(case when acct_cls_flg=\'N\' then 1 else 0 end) FD_ACTV_CNT, \
SUM(CLR_BAL_AMT) FD_AMOUNT_SUM \
from tbaadm.gam@stagelink a \
left join DWH.dim_dayend_product b on a.schm_code=b.schm_code \
where lower(product_sub_type) like \'%fixed%\' \
GROUP BY a.cust_id'

odQuery='select a.cust_id, \
sum(case when acct_cls_flg=\'Y\' then 1 else 0 end) ODA_CLS_CNT, \
sum(case when acct_cls_flg=\'N\' then 1 else 0 end) ODA_ACTV_CNT, \
SUM(CLR_BAL_AMT) ODA_AMOUNT_TOT \
from tbaadm.gam@stagelink a \
WHERE SCHM_TYPE=\'ODA\' \
GROUP BY a.cust_id'

loanQuery='select A.CUST_ID, \
SUM(case when acct_cls_flg=\'Y\' then 1 else 0 end) INACTV_LOAN_CNT, \
SUM(case when acct_cls_flg=\'N\' then 1 else 0 end) ACTV_LOAN_CNT, \
SUM(case when acct_cls_flg=\'Y\' then a.SANCT_LIM else 0 end) INACTV_LOAN_LIM_TOT, \
SUM(case when acct_cls_flg=\'N\' then a.SANCT_LIM else 0 end) ACTV_LOAN__LIM_TOT, \
SUM(CLR_BAL_AMT) AS TOT_BAL, \
MAX(a.SANCT_LIM) MAX_LOAN_LIM, \
MIN(a.SANCT_LIM) MIN_LOAN__LIM, \
(LAST_DAY(add_months(trunc(sysdate,\'mm\'),-1))-MAX(a.ACCT_OPN_DATE)) LAST_LOAN_AGE_DAYS, \
(LAST_DAY(add_months(trunc(sysdate,\'mm\'),-1))-MIN(a.ACCT_OPN_DATE)) FIRST_LOAN_AGE_DAYS, \
sum(case when lower(our_product_group) like \'%staff%\' then 1 else 0 end) STAFF_LOAN_CNT, \
sum(case when lower(our_product_group) like \'%short%\' then 1 else 0 end) ST_LOAN_CNT, \
sum(case when lower(our_product_group) like \'%medium%\' then 1 else 0 end) MT_LOAN_CNT, \
sum(case when lower(our_product_group) like \'%long%\' then 1 else 0 end) LT_LOAN_CNT, \
sum(case when lower(our_product_group) like \'%housing%\' then 1 else 0 end) HOUSING_LOAN_CNT \
from tbaadm.gam@stagelink a \
inner join (select distinct schm_code,our_product_group from stg.f10_ccu_adv_portfolio) b on a.schm_code=b.schm_code \
and lower(b.our_product_group) like \'%loan%\' \
group by a.cust_id'

leaseQuery='select A.CUST_ID, \
SUM(case when acct_cls_flg=\'Y\' then 1 else 0 end) INACTV_LEASE_CNT, \
SUM(case when acct_cls_flg=\'N\' then 1 else 0 end) ACTV_LEASE_CNT, \
SUM(case when acct_cls_flg=\'Y\' then a.SANCT_LIM else 0 end) INACTV_LEASE_LIM_TOT, \
SUM(case when acct_cls_flg=\'N\' then a.SANCT_LIM else 0 end) ACTV_LEASE__LIM_TOT, \
SUM(CLR_BAL_AMT) AS TOT_BAL, \
MAX(a.SANCT_LIM) MAX_LEASE_LIM, \
MIN(a.SANCT_LIM) MIN_LEASE__LIM, \
(LAST_DAY(add_months(trunc(sysdate,\'mm\'),-1))-MAX(a.ACCT_OPN_DATE)) LAST_LEASE_AGE_DAYS, \
(LAST_DAY(add_months(trunc(sysdate,\'mm\'),-1))-MIN(a.ACCT_OPN_DATE)) FIRST_LEASE_AGE_DAYS \
from tbaadm.gam@stagelink a \
inner join (select distinct schm_code,our_product_group from stg.f10_ccu_adv_portfolio) b on a.schm_code=b.schm_code \
and lower(b.our_product_group) like \'%lease%\' \
group by a.cust_id'

ccQuery='select distinct lower(p.ssnumber) as NIC_NUMBER,max(a.creditlimit) as CreditLimit \
from tbaadm.cardx c \
left join tbaadm.ctransactions ct on c.serno=ct.cardserno \
inner join tbaadm.people p on c.peopleserno=p.serno \
inner join tbaadm.caccounts a on c.caccserno=a.serno \
group by lower(p.ssnumber)'

fbQuery='select distinct cust_id \
from tbaadm.gam \
where schm_code=\'VIPAF\''

crmQuery='select \
CORE_CUST_ID as CUST_ID,UNIQUEIDNUMBER AS CRM_NIC1,UNIQUEID CRM_NIC2,CUST_lAST_NAME AS CRM_CUST_NAME,GENDER AS CRM_GENDER, \
(EXTRACT(YEAR FROM SYSDATE)-BIRTH_YEAR) CRM_AGE,OCCUPATION CRM_OCCUPATION, \
PHONE CRM_PHONE1,PHONE_HOME CRM_PHONE2,PHONE_HOME2 CRM_PHONE3,PHONE_CELL CRM_PHONE4,PREFERREDPHONE CRM_PHONE5 , EMAIL CRM_EMAIL1, \
EMAIL_PALM CRM_EMAIL2,EMAIL_HOME CRM_EMAIL3,PRIMARY_SOL_ID CRM_BRANCH_ID,CONCAT(CONCAT(ADDRESS_LINE1,ADDRESS_LINE2), \
ADDRESS_LINE3) AS CRM_ADDRESS \
,STATE AS CRM_STATE \
from CRMUSER.ACCOUNTS'

vQuery='select USER_ID as USER_ID,NIC_NUMBER AS V_NIC_NUMBER,PP_NUMBER AS V_PP_NUMBER,EMAIL_ADDR AS V_EMAIL, \
CONCAT(CUST_OTHER_NAMES , CUST_SURNAME) AS V_CUST_NAME,CONCAT(COMM_ADDR1,COMM_ADDR2) AS V_ADDRESS, \
COMM_TEL_NO AS V_PHONE1,COMM_MOBILE_NO AS V_PHONE2 \
from tbaadm.svr_user_master_table'

fbAppQuery='' &=******************** have to fill ******************************

fbGAMQuery='select ditinct CUST_ID as CUST_ID, 1 as feedback from tbaadm.gam where schm_code=\'VIPAF\' '


connection_182=dbConnection('anlinq/anlinq@172.16.1.182:1521/dwh')
connection_172=dbConnection('anlinq/anlinq@172.16.1.172:1521/BIDB')

vishwaCustomer=get_raw_data(connection_182,vishCustomer,"vishwa customer data extraction started",
                                      "vishwa customer data extraction completed")

vishwaTransfer=get_raw_data(connection_182,vishTransfer,"vishwa transfer data extraction started",
                                      "vishwa transfer data extraction completed")

vishwaPayment=get_raw_data(connection_182,vishPayment,"vishwa payment data extraction started",
                                      "vishwa payment data extraction completed")


vishwaSchdPayment=get_raw_data(connection_182,vishSchdPayment,"vishwa schedule payment data extraction started",
                                      "vishwa schedule payment data extraction completed")

userCustTrack=get_raw_data(connection_182,userCustMap,"vishwa user cust data extraction started",
                                   "vishwa user cust data extraction completed")

customer=get_raw_data(connection_182,custQuery," Customer data extraction started",
                                      " Customer data extraction completed")

pawnCustomer=get_raw_data(connection_182,pawnCustomer," pawn Customer data extraction started",
                                 " pawn Customer data extraction completed",)

txnATMSum=get_raw_data(connection_172,txnATMQuery," ATM data extraction started",
                                  " ATM data extraction completed")

txnPOSSum=get_raw_data(connection_172,txnPOSQuery,"POS data extraction started",
                                  " POS data extraction completed")


fd=get_raw_data(connection_172,fdQuery,"FD data extraction started","FD data extraction completed")

oda=get_raw_data(connection_172,odQuery,"ODA data extraction started","ODA data extraction completed")

loan=get_raw_data(connection_172,loanQuery,"loan data extraction started","loan data extraction completed")

lease=get_raw_data(connection_172,loanQuery,"lease data extraction started","lease data extraction completed")

cc=get_raw_data(connection_182,ccQuery,"credit card data extraction started","credit data extraction completed")

fb=get_raw_data(connection_182,fbQuery,"Already vishwa loan taken users data extraction started",
                        "Already vishwa loan taken users data extraction completed")

crmUser=get_raw_data(connection_182,crmQuery,"Customer demographic data from crmuseraccounts extraction started",
                     "Customer demograhic data from crmuseraccounts data extraction completed")

vUser=get_raw_vUser(connection_182,vQuery,"Customer demographic data from vishwa table extraction started",
                     "Customer demograhic data from vishwa table data extraction completed")

fbGAM=get_raw_data(connection_182,fbGAMQuery,"Feedback GAM data extraction started"
                   ,"Feedback GAM data extraction completed")

fbApp=********************************Have to fill **********************************


vishwaDF = vishwaCustomer.merge(right=vishwaTransfer.reset_index(drop=True), how='left', on='USER_ID')
vishwaDF = vishwaDF.merge(right=vishwaPayment.reset_index(drop=True), how='left', on='USER_ID')
vishwaDF = vishwaDF.merge(right=vishwaSchdPayment.reset_index(drop=True), how='left', on='USER_ID')

df0=userCustTrack[(userCustTrack.U_CNT==1) & (userCustTrack.C_CNT==1)]
df = vishwaDF.merge(right=df0.reset_index(drop=True), how='inner', on='USER_ID')
df1 = df.merge(right=customer.reset_index(drop=True), how='left', on='CUST_ID')
df2=df1.merge(right=pawnCustomer.reset_index(drop=True), how='left', left_on='UNIQUEIDNUMBER',right_on='P_NIC_NUMBER')
df3=df2.merge(right=txnATMSum.reset_index(drop=True), how='left', left_on='CUST_ID',right_on='CUST_ID')
df4=df3.merge(right=txnPOSSum.reset_index(drop=True), how='left', left_on='CUST_ID',right_on='CUST_ID')
df5=df4.merge(right=fd.reset_index(drop=True), how='left', left_on='CUST_ID',right_on='CUST_ID')
df6=df5.merge(right=oda.reset_index(drop=True), how='left', left_on='CUST_ID',right_on='CUST_ID')
df7=df6.merge(right=loan.reset_index(drop=True), how='left', left_on='CUST_ID',right_on='CUST_ID')
df8=df7.merge(right=transaction.reset_index(drop=True),how='left', left_on='CUST_ID',right_on='CUST_ID')
df9=df8.merge(right=lease.reset_index(drop=True), how='left', left_on='CUST_ID',right_on='CUST_ID')
df10=df9.merge(right=cc.reset_index(drop=True), how='left', left_on='UNIQUEIDNUMBER',right_on='NIC_NUMBER')
application = df10.merge(right=fb.reset_index(drop=True), how='left', on='CUST_ID')
df12 = vUser.merge(right=df0.reset_index(drop=True), how='inner',on='USER_ID')
custDemographic= df12.merge(right=crmUser.reset_index(drop=True), how='left', on='CUST_ID')
feedback = pd.concat(fbGAM,fbApp,axis=0)
data=application.merge(right=feedback.reset_index(drop=True), how='left', on='CUST_ID')

imputeMissing = {
#    'BANK_CUST_FLG',
    'CREDIT_CARD_FLAG':0,
    'V_COUNTRY_SL':'NA',
#    'VISHWA_AGE',
#    'LAST_LOGIN_AGE',
    'V_OT_TXN_AMOUNT_LKR':0,
    'V_RA_TXN_AMOUNT_LKR':0,
    'V_RT_TXN_AMOUNT_LKR':0,
    'V_MC_TXN_AMOUNT_LKR':0,
    'V_TT_TXN_AMOUNT_LKR':0,
    'V_TXN_CNT':0,
    'V_PMNT_TRANSACTION_AMT_LKR':0,
    'V_PMNT_COUNT':0,
    'V_SCH_PMNT_AMOUNT_LKR':0,
    'V_SCH_PMNT_CNT':0,
#    'CUST_ID',
    'GENDER':'NA',
    'STAFFFLAG':'NA',
    'CARD_HOLDER':'NA',
#    'CUST_AGE',
    'CUST_TYPE':'NA',
    'OCCUPATION':'NA',
    'CITY':'NA',
#    'BANK_CUST_AGE_DAYS',
    'SENIORCITIZEN':'NA',
    'BLACKLISTED':'NA',
    'NEGATED':'NA',
    'SUSPENDED':'NA',
    'PTSTATUS_R':0,
    'PTSTATUS_A':0,
    'PTSTATUS_C':0,
    'PTVALUE':0,
    'ATM_TRXN_BILL_AMT':0,
    'POS_TRXN_BILL_AMT':0,
    'FD_CLS_CNT':0,
    'FD_ACTV_CNT':0,
    'FD_AMOUNT_SUM':0,
    'ODA_CLS_CNT':0,
    'ODA_ACTV_CNT':0,
    'ODA_AMOUNT_TOT':0,
    'INACTV_LOAN_CNT':0,
    'ACTV_LOAN_CNT':0,
    'INACTV_LOAN_LIM_TOT':0,
    'ACTV_LOAN__LIM_TOT':0,
    'TOT_BAL_x':0,
    'MAX_LOAN_LIM':0,
    'MIN_LOAN__LIM':0,
    'LAST_LOAN_AGE_DAYS':0,
    'FIRST_LOAN_AGE_DAYS':0,
    'STAFF_LOAN_CNT':0,
    'ST_LOAN_CNT':0,
    'MT_LOAN_CNT':0,
    'LT_LOAN_CNT':0,
    'HOUSING_LOAN_CNT':0,
    'C':0,
    'D':0,
    'cpayFlag':0,
    'INACTV_LEASE_CNT':0,
    'ACTV_LEASE_CNT':0,
    'INACTV_LEASE_LIM_TOT':0,
    'ACTV_LEASE__LIM_TOT':0,
    'TOT_BAL_y':0,
    'MAX_LEASE_LIM':0,
    'MIN_LEASE__LIM':0,
    'LAST_LEASE_AGE_DAYS':0,
    'FIRST_LEASE_AGE_DAYS':0,
    'CREDITLIMIT':0
}

data.fillna(value=imputeMissing,inplace=True)
data = data.query('(Target >=0) or (CUST_AGE<=50 and cpayFlag >= 3 and ACTV_LOAN_CNT ==0 and ACTV_LEASE_CNT ==0 and BANK_CUST_FLG == "Y" and V_COUNTRY_SL == "Yes" and STAFFFLAG =="N" and BLACKLISTED=="N")')

data.loc[:,'LAST_LOGIN_AGE'] = data.LAST_LOGIN_AGE 
data['cc_flag'] = np.where(data['CREDITLIMIT']>0, 1, 0)
data['vishwaTxAmt'] =     data.V_OT_TXN_AMOUNT_LKR + data.V_RA_TXN_AMOUNT_LKR + data.V_RT_TXN_AMOUNT_LKR + data.V_MC_TXN_AMOUNT_LKR \
                     + data.V_TT_TXN_AMOUNT_LKR + data.V_PMNT_TRANSACTION_AMT_LKR + data.V_SCH_PMNT_AMOUNT_LKR
data['vishwaTxCnt'] =     data.V_SCH_PMNT_CNT + data.V_PMNT_COUNT + data.V_SCH_PMNT_CNT
data['ttlLoanCnt'] = data.ST_LOAN_CNT + data.MT_LOAN_CNT + data.HOUSING_LOAN_CNT
data['Dr2Cr'] = data.D/data.C
data['ATM_TRXN_BILL_AMT2Dr'] = data.ATM_TRXN_BILL_AMT/data.D

remList=['CUST_ID','V_CUST_DOB','BANK_CUST_FLG','CREDIT_CARD_FLAG','V_COUNTRY_SL','V_RT_TXN_AMOUNT_LKR','CARD_HOLDER',
         'CUST_TYPE','OCCUPATION','CITY','SENIORCITIZEN','NEGATED','SUSPENDED','PTSTATUS_C','ACTV_LOAN_CNT',
         'ACTV_LOAN__LIM_TOT','TOT_BAL_x','LT_LOAN_CNT','INACTV_LEASE_CNT','ACTV_LEASE_CNT','INACTV_LEASE_LIM_TOT',
         'cpayFlag','ACTV_LEASE__LIM_TOT','TOT_BAL_y','MAX_LEASE_LIM','MIN_LEASE__LIM','LAST_LEASE_AGE_DAYS','FIRST_LEASE_AGE_DAYS',
         'CREDITLIMIT','STAFFFLAG','BLACKLISTED','BLACKLISTED','V_MC_TXN_AMOUNT_LKR','Target']

data = transform_df(data)
data = pd.get_dummies(data,columns=['GENDER'],drop_first=True)

featureList = [ 'VISHWA_AGE',
                'LAST_LOGIN_AGE',
                #'V_OT_TXN_AMOUNT_LKR',
                #'V_RA_TXN_AMOUNT_LKR',
                #'V_TT_TXN_AMOUNT_LKR',
                #'V_TXN_CNT',
                #'V_PMNT_TRANSACTION_AMT_LKR',
                #'V_PMNT_COUNT',
                #'V_SCH_PMNT_AMOUNT_LKR',
                #'V_SCH_PMNT_CNT',
                'GENDER_M',
                'CUST_AGE',
                'BANK_CUST_AGE_DAYS',
                'PTSTATUS_R',
                'PTSTATUS_A',
                'PTVALUE',
                'ATM_TRXN_BILL_AMT',
                'POS_TRXN_BILL_AMT',
                'FD_CLS_CNT',
                'FD_ACTV_CNT',
                #'FD_AMOUNT_SUM',
                'ODA_CLS_CNT',
                'ODA_ACTV_CNT',
                #'ODA_AMOUNT_TOT',
                'INACTV_LOAN_CNT',
                #'INACTV_LOAN_LIM_TOT',
                #'MAX_LOAN_LIM',
                #'MIN_LOAN__LIM',
                'LAST_LOAN_AGE_DAYS',
                'FIRST_LOAN_AGE_DAYS',
                #'STAFF_LOAN_CNT',
                'ST_LOAN_CNT',
                'MT_LOAN_CNT',
                'HOUSING_LOAN_CNT',
                'C',
                'D',
                'cc_flag',
                'vishwaTxAmt',
                'vishwaTxCnt',
                #'ttlLoanCnt',
                'Dr2Cr',
                'ATM_TRXN_BILL_AMT2Dr']


featureList = [e for e in list(data) if e not in remList]
data[featureList] = data[featureList].fillna(data[featureList].mean())
X = data.loc[~data.Target.isna(),featureList]
Y = data[~data.Target.isna()].Target
#X.fillna(X.mean(),inplace=True)

X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size = 0.3, random_state = 100)
rf = RandomForestClassifier(n_estimators=10,max_depth=2,bootstrap=True,random_state=80,max_leaf_nodes=2,max_features=5)
rf = rf.fit(X_train, y_train)

colList=['Id'
        ,'Name'
        ,'ContactNo'
        ,'NIC'
        ,'GENDER'
        ,'AGE'
        ,'PHONE1'
        ,'PHONE2'
        ,'PHONE3'
        ,'PHONE4'
        ,'BRCH'
        ,'BRCHNM'
        ,'ADDRESS1'
        ,'ADDRESS2'
        ,'ADDRESS3'
        ,'ADDRESS4'
        ,'ADDRESS5'
        ,'ADDRESS6'
        ,'EMAIL1'
        ,'EMAIL2'
        ,'CR'
        ,'DR'
        ,'CIF'
        ,'BatchCode'
        ,'CurrentBatch'
        ,'Calling'
        ,'CallingTime'
        ,'CallingUser'
        ,'External']

predDf = pd.DataFrame(columns=colList)


